# Test unitaire - Activité 3

---
Nom : Chibane

Prénom : Ilies

Groupe de TP : IA2S & GP-IA

---

> ☝ **Objectif :** Publier sous forme textuelle, dans une page Web, les résultats de l'analyse des données.


## Préambule

Avant toute chose, il faut importer les bibliothèques et modules utiles :

> 🛑 ne pas modifier le code donné ci-dessous.

In [ ]:
!pip install contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 77.5 MB/s eta 0:00:00


In [ ]:
# importations des bibliothèques utiles et configuration
#

from google.colab import drive # pour l'accès au drive
import sys                     # pour l'accès au commandes système
import pandas as pd            # pour les fonctionnalités du module pandas

# montage du drive sur Colab (avec autorisation d'accès)
drive.mount('/content/drive', force_remount=True)
# cliquez sur le lien affiché dans la console afin d'obtenir votre clé d'accès
# et recopiez cette clé dans la zone de saisie de texte.

# pour trouver les fonctions supports "faites maison" et spécifiques
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/SAE15/tools')
import sae15_tools as tools # fonctions supports
import sae15_spec as spec   # fonctions spécifiques

Mounted at /content/drive


## Lecture des données Vélib

In [ ]:
# chargement des données de status et d'information depuis l'API Vélib' Métropole (fonctions supports)
#
json_data_info = tools.loadVelibInformation()
json_data_statu = tools.loadVelibStatus()

## Préparation des données

In [ ]:
# conversion en Data Frames Pandas des données de status et d'information (pd.DataFrame)
info_stations = tools.getVelibStations(json_data_info)
df_info_stations = pd.DataFrame.from_dict(info_stations)
statu_stations = tools.getVelibStations(json_data_statu)
df_statu_stations = pd.DataFrame.from_dict(statu_stations)
# fusion des deux Data Frames (fonction merge)
merged_df = pd.merge(df_info_stations, df_statu_stations, on=['station_id', 'stationCode'])
# affichage du résultat
merged_df

,station_id,name,lat,lon,capacity,stationCode,rental_methods,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN,5,5,"[{'mechanical': 1}, {'ebike': 4}]",29,29,1,1,1,1700342045
1,653222953,Mairie de Rosny-sous-Bois,48.871257,2.486581,30,31104,[CREDITCARD],28,28,"[{'mechanical': 9}, {'ebike': 19}]",2,2,1,1,1,1700341778
2,17278902806,Rouget de L'isle - Watteau,48.778193,2.396302,20,44015,NaN,11,11,"[{'mechanical': 2}, {'ebike': 9}]",8,8,1,1,1,1700342179
3,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD],7,7,"[{'mechanical': 1}, {'ebike': 6}]",13,13,1,1,1,1700341880
4,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN,25,25,"[{'mechanical': 21}, {'ebike': 4}]",5,5,1,1,1,1700342151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,54000604,Ordener - Poissonniers,48.891214,2.351289,35,18023,[CREDITCARD],2,2,"[{'mechanical': 1}, {'ebike': 1}]",30,30,1,1,1,1700342068
1467,43247738,Ruisseau - Ordener,48.892995,2.340145,35,18026,[CREDITCARD],6,6,"[{'mechanical': 1}, {'ebike': 5}]",24,24,1,1,1,1700342186
1468,102311820,Clignancourt - Ordener,48.891458,2.348636,35,18024,[CREDITCARD],4,4,"[{'mechanical': 0}, {'ebike': 4}]",31,31,1,1,1,1700341609
1469,368766689,Westermeyer - Paul Vaillant-Couturier,48.819116,2.396664,25,42004,[CREDITCARD],32,32,"[{'mechanical': 26}, {'ebike': 6}]",0,0,1,1,1,1700342079


## Analyses statistiques

In [ ]:
# mesures statistiques assemblées (fonctions describe et join)
from pandas.api.types import is_numeric_dtype
df_stat = []

for d in merged_df:
  if is_numeric_dtype(merged_df[d]) and d not in ['station_id', 'stationCode', 'last_reported']:
    stats = spec.stationStatistics(merged_df[d])
    row = {"Colonne":d}
    row.update(stats)
    df_stat.append(row)

df_stat = pd.DataFrame.from_dict(df_stat)

df_stat

,Colonne,mean,count,std,min,max
0,lat,48.858439,1471,0.030799,48.764615,48.951432
1,lon,2.341008,1471,0.056640,2.165597,2.538242
2,capacity,31.286200,1471,13.516946,0.000000,200.000000
3,num_bikes_available,10.662814,1471,10.182109,0.000000,64.000000
4,numBikesAvailable,10.662814,1471,10.182109,0.000000,64.000000
5,num_docks_available,19.380014,1471,14.011914,0.000000,200.000000
6,numDocksAvailable,19.380014,1471,14.011914,0.000000,200.000000
7,is_installed,0.992522,1471,0.086180,0.000000,1.000000
8,is_returning,0.965330,1471,0.183006,0.000000,1.000000
9,is_renting,0.965330,1471,0.183006,0.000000,1.000000


## Exportation pour le Web

In [ ]:
# renommage des colonnes avant l'exportation en HTML (fonction rename)
df_stat.rename(columns={
    'Colonne': 'Nom_de_la_Colonne',
    'mean': 'moyenne',
    'count': 'nombre',
    'std': 'ecart_type',
    'min': 'minimum',
    'max': 'maximum'
}, inplace=True)

# exportation des mesures au format HTML (fonctions open et to_html)
spec.exportStatistics(df_stat, "stats.html")

---
🎯 Livrer ce test unitaire sur Eprel en fin de séance

---



